#RNN

In this notebook I tried to solve the classification model with an RNN.

## TODO:
- [ ] Stemming
- [x] Remove Stop Words


In [1]:
import pandas as pd

C:\Users\Ayham\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\Ayham\anaconda3\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
C:\Users\Ayham\anaconda3\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [3]:
df = pd.read_csv("./dataset.csv")
df

,text,humor
0,"Joe biden rules out 2020 bid: 'guys, i'm not r...",False
1,Watch: darvish gave hitter whiplash with slow ...,False
2,What do you call a turtle without its shell? d...,True
3,5 reasons the 2016 election feels so personal,False
4,"Pasco police shot mexican migrant from behind,...",False
...,...,...
199995,Conor maynard seamlessly fits old-school r&b h...,False
199996,How to you make holy water? you boil the hell ...,True
199997,How many optometrists does it take to screw in...,True
199998,Mcdonald's will officially kick off all-day br...,False


In [4]:
df.describe()

,text,humor
count,200000,200000
unique,200000,2
top,"Joe biden rules out 2020 bid: 'guys, i'm not r...",False
freq,1,100000


# Check for imbalance

The Dataset is equally balance, no need for a rebalance

In [5]:
df["humor"].value_counts()

False    100000
True     100000
Name: humor, dtype: int64

# Preprocessing

In [6]:
# Transform class from Boolean to integer value
df['class']=df['humor'].apply(lambda x: 1 if x==True else 0)

In [8]:
#remove stop words
from gensim.parsing.preprocessing import remove_stopwords
df["text"] = df["text"].apply(lambda x: remove_stopwords(x))

In [9]:
df

,text,humor,class
0,"Joe biden rules 2020 bid: 'guys, i'm running'",False,0
1,Watch: darvish gave hitter whiplash slow pitch,False,0
2,What turtle shell? dead.,True,1
3,5 reasons 2016 election feels personal,False,0
4,"Pasco police shot mexican migrant behind, new ...",False,0
...,...,...,...
199995,Conor maynard seamlessly fits old-school r&b h...,False,0
199996,How holy water? boil hell,True,1
199997,How optometrists screw lightbulb? one... two? ...,True,1
199998,Mcdonald's officially kick all-day breakfast o...,False,0


In [12]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df['text'],df['class'], stratify=df['class'])

ImportError: cannot import name 'suppress_warnings' from 'numpy.testing' (unknown location)

In [ ]:
dataset = pd.read_csv("dataset.csv")
#train_dataset, test_dataset = dataset['train'], dataset['test']
dataset
#train_dataset.element_spec